### Entegra Report Analysis

In [88]:
# pip install geopy

In [89]:
import unicodedata
import datetime as dt
import os
import sys
import re
import numpy as np
import pandas as pd
import math
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import locale
from locale import atof
from geopy.geocoders import Nominatim
from geopy.point import Point
from geopy.exc import GeocoderTimedOut
from functools import reduce

##### Functions

In [90]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [91]:
def convert_one_character_letter(text):
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [92]:
def clean_text(text, custom_alp=tr): # for only string in specific language
    '''This function extract custom_alp character from string \n
    clean_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.
    '''
    # text_clean = re.findall(custom_alp, text)
    text_result_list = []
    text_list = re.findall(r'\S+', text)
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_result = " ".join(text_result_list)
    return text_result

In [93]:
def lower_func(text):
    '''This function convert string character to lowercase. \n
    lower_func(text=str): text is any string word or sentence
    '''
    string_lower = str.lower(text)
    return string_lower

In [94]:
def upper_func(text):
    '''This function convert string character to uppercase. \n
    upper_func(text=str): text is any string word or sentence
    '''
    string_upper = str.upper(text)
    return string_upper

In [95]:
def capitalize_func(text):
    '''This function convert string character to capitalize case. \n
    capitalize_func(text=str): text is any string word or sentence
    '''
    string_capitalize = str.capitalize(text)
    return string_capitalize

In [96]:
def title_func(text):
    '''This function convert string character to title as each word start with capital letter. \n
    title_func(text=str): text is any string word or sentence
    '''
    string_title = str.title(text)
    return string_title

In [97]:
def character_clean_lower_text(text, custom_alp=tr): # for only string in specific language
    '''This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. And also extract custom_alp character from string and convert to lowercase. \n
    character_clean_lower_text(text, custom_alp): text is any string word or sentence and custom_alp is specific alphabet.  
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    text_result_list = []
    text_list = re.findall(r'\S+', new_text)  # like as word tokenize
    for text_var in text_list:
        text_clean = re.findall(custom_alp, str(text_var))
        text_result_var = "".join(text_clean)
        text_result_list.append(text_result_var)
    text_join = " ".join(text_result_list)
    text_result = str.lower(text_join)
    return text_result

In [98]:
def convert_to_float(value):
    '''This function convert string numeric value to float type. Like as 1.200,15
    convert_to_float(value): value is a string numeric value
    '''
    value = value.replace(".","").replace(",",".")
    return float(value)

In [99]:
def whitespace_del(text):
    '''This function provides removing left right white space.\n
    whitespace_del(text): text is a string value as word or sentence.
    '''
    text_var = str(text)
    text_var = text_var.strip()  # rstrip lstrip
    return text_var

#### Report

In [100]:
variable_date = "01.04.2024-05.09.2024"
input_path = fr"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Entegra\Sales\{variable_date}"
output_path = fr"C:\Users\user\Downloads"

data_file = fr"Entegra_Sales_List ({variable_date})"

In [101]:
# df_entegra = pd.read_excel(fr"{input_path}\{data_file}.xlsx")

# df_entegra.to_csv(fr"{output_path}\{data_file}.csv", encoding='utf8', index=False)

In [102]:
#df_entegra = pd.read_csv(fr"{input_path}\{data_file}.csv", low_memory=False)
df_entegra = pd.read_excel(fr"{input_path}\{data_file}.xlsx")
df_entegra

,id,order_number,platform_reference_no,invoice_number,datetime,date_add,entegration,supplier,status,status_name,...,pov_productCode,payment_type,total_product,PazaryerindenGelenOdemeTutar,total_product_quantity,delivery_method,fatura_tarihi,invoice_url,invoice_type,FaturaTipiAdi
0,201036,2383110310,9465795799,NaN,2024-09-05 10:54:27,2024-09-05 10:57:15,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
1,201035,2383102583,9465789564,NaN,2024-09-05 10:50:42,2024-09-05 10:51:14,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,5,0,7,Aynı gün kargoda,NaN,NaN,0,Seciniz
2,201033,2383091113,9465777317,NaN,2024-09-05 10:43:15,2024-09-05 10:45:03,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
3,201025,2383085464,9465772854,NaN,2024-09-05 10:40:26,2024-09-05 10:42:12,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,2,0,2,Aynı gün kargoda,NaN,NaN,0,Seciniz
4,201027,2383076067,9465764076,NaN,2024-09-05 10:34:55,2024-09-05 10:42:15,trendyol,trendyol,2,Onaylandı,...,220647971025,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,107816,2036971893,9080057915,NaN,2024-03-01 00:16:54,2024-03-01 00:17:34,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,03/01/2024 10:59:38,NaN,0,Seciniz
92976,107817,2036970386,9080056913,NaN,2024-03-01 00:16:07,2024-03-01 00:17:35,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,03/01/2024 09:21:47,NaN,0,Seciniz
92977,107815,0888556689,0888556689,NaN,2024-03-01 00:14:02,2024-03-01 00:15:22,HepsiBurada,hb,2,Onaylandı,...,NaN,HepsiBurada,1,0,1,NaN,03/01/2024 09:53:35,NaN,0,Seciniz
92978,107812,2036956451,9080042398,NaN,2024-03-01 00:05:22,2024-03-01 00:06:28,trendyol,trendyol,2,Onaylandı,...,yatak54896513739,Trendyol,1,0,1,NaN,03/01/2024 10:59:38,NaN,0,Seciniz


In [103]:
df_entegra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92980 entries, 0 to 92979
Data columns (total 59 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            92980 non-null  int64         
 1   order_number                  92980 non-null  object        
 2   platform_reference_no         92980 non-null  object        
 3   invoice_number                0 non-null      float64       
 4   datetime                      92980 non-null  datetime64[ns]
 5   date_add                      92980 non-null  datetime64[ns]
 6   entegration                   92980 non-null  object        
 7   supplier                      92980 non-null  object        
 8   status                        92980 non-null  int64         
 9   status_name                   92980 non-null  object        
 10  company                       92979 non-null  object        
 11  cargo_code                  

In [104]:
df_entegra.columns = [x.lower() for x in df_entegra.columns]
df_entegra.columns = [x.strip() for x in df_entegra.columns]
df_entegra

,id,order_number,platform_reference_no,invoice_number,datetime,date_add,entegration,supplier,status,status_name,...,pov_productcode,payment_type,total_product,pazaryerindengelenodemetutar,total_product_quantity,delivery_method,fatura_tarihi,invoice_url,invoice_type,faturatipiadi
0,201036,2383110310,9465795799,NaN,2024-09-05 10:54:27,2024-09-05 10:57:15,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
1,201035,2383102583,9465789564,NaN,2024-09-05 10:50:42,2024-09-05 10:51:14,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,5,0,7,Aynı gün kargoda,NaN,NaN,0,Seciniz
2,201033,2383091113,9465777317,NaN,2024-09-05 10:43:15,2024-09-05 10:45:03,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
3,201025,2383085464,9465772854,NaN,2024-09-05 10:40:26,2024-09-05 10:42:12,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,2,0,2,Aynı gün kargoda,NaN,NaN,0,Seciniz
4,201027,2383076067,9465764076,NaN,2024-09-05 10:34:55,2024-09-05 10:42:15,trendyol,trendyol,2,Onaylandı,...,220647971025,Trendyol,1,0,1,Aynı gün kargoda,NaN,NaN,0,Seciniz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,107816,2036971893,9080057915,NaN,2024-03-01 00:16:54,2024-03-01 00:17:34,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,03/01/2024 10:59:38,NaN,0,Seciniz
92976,107817,2036970386,9080056913,NaN,2024-03-01 00:16:07,2024-03-01 00:17:35,trendyol,trendyol,2,Onaylandı,...,NaN,Trendyol,1,0,1,Ertesi Gün Kargoda,03/01/2024 09:21:47,NaN,0,Seciniz
92977,107815,0888556689,0888556689,NaN,2024-03-01 00:14:02,2024-03-01 00:15:22,HepsiBurada,hb,2,Onaylandı,...,NaN,HepsiBurada,1,0,1,NaN,03/01/2024 09:53:35,NaN,0,Seciniz
92978,107812,2036956451,9080042398,NaN,2024-03-01 00:05:22,2024-03-01 00:06:28,trendyol,trendyol,2,Onaylandı,...,yatak54896513739,Trendyol,1,0,1,NaN,03/01/2024 10:59:38,NaN,0,Seciniz


In [105]:
df_entegra.columns

Index(['id', 'order_number', 'platform_reference_no', 'invoice_number',
       'datetime', 'date_add', 'entegration', 'supplier', 'status',
       'status_name', 'company', 'cargo_code', 'invoice_country',
       'invoice_city', 'tax_office', 'tax_number', 'tc_id', 'total', 'tax',
       'grand_total', 'sync', 'erp_order_number', 'erp_message',
       'store_order_status', 'store_order_status_name', 'order_process',
       'note', 'supplier1', 'supplier_id', 'cargo_sync', 'cargo_company',
       'cargo_code2', 'cargo_follow_url', 'cargo_finally_statu',
       'cargo_error_message', 'einvoice_error_message', 'invoice_print_name',
       'ship_print_name', 'sevk_tarihi', 'ship_country', 'customer_code',
       'cargo_fee_type_name', 'cargo_packet_type', 'cargo_packet_quantity',
       'grand_total_desi', 'model', 'product_name', 'store_product_name',
       'invoice_name', 'pov_productcode', 'payment_type', 'total_product',
       'pazaryerindengelenodemetutar', 'total_product_quantity',

In [106]:
df_entegra.id.nunique()

92980

In [107]:
df_entegra.model.nunique()

974

In [108]:
df_entegra.pov_productcode.nunique()  # barcode

372

In [109]:
df_entegra["invoice_country"].unique()

array([nan, 'Türkiye', 'Birleşik Arap Emirlikleri', 'Romanya',
       'Azerbaycan', 'Suudi Arabistan', 'Slovakya', 'Kuveyt', 'Katar',
       'Umman', 'Bahreyn'], dtype=object)

In [110]:
df_entegra_select_var = df_entegra[["order_number","model","pov_productcode","product_name","datetime","total_product_quantity","entegration",\
    "invoice_city","total","tax","grand_total","company","cargo_company","status_name"]]
df_entegra_select_var

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,391.670,78.330,470.00,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,395.530,48.470,444.00,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,165.830,33.170,199.00,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,215.830,43.170,259.00,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,181.820,18.180,200.00,Damla Beşer,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,2036971893,001603,NaN,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,trendyol,Samsun,133.330,26.670,160.00,Gülizar Sarı,trendyolekspress,Onaylandı
92976,2036970386,002429,NaN,BTS3 Boyama Tuval Sayılı Set 25x35 Yunus,2024-03-01 00:16:07,1,trendyol,Çorum,154.170,30.830,185.00,zübeyde DEMİREL,trendyolekspress,Onaylandı
92977,0888556689,002369,NaN,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,HepsiBurada,İzmir,33.325,6.665,39.99,İrem Erbey,hepsijet,Onaylandı
92978,2036956451,002284,yatak54896513739,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,trendyol,Konya,142.500,28.500,171.00,Esma Söylemez,trendyolekspress,Onaylandı


In [111]:
df_entegra_select_var["cargo_company"].unique()

array(['trendyolekspress', 'mng', 'hepsijet', 'eptt', 'yurtici',
       'kolaygelsin', 'horoz', 'surat', nan], dtype=object)

In [112]:
df_entegra_select_var.loc[:,"cargo_company"] = df_entegra_select_var.loc[:,"cargo_company"].fillna("diger")
df_entegra_select_var

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,391.670,78.330,470.00,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,395.530,48.470,444.00,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,165.830,33.170,199.00,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,215.830,43.170,259.00,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,181.820,18.180,200.00,Damla Beşer,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,2036971893,001603,NaN,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,trendyol,Samsun,133.330,26.670,160.00,Gülizar Sarı,trendyolekspress,Onaylandı
92976,2036970386,002429,NaN,BTS3 Boyama Tuval Sayılı Set 25x35 Yunus,2024-03-01 00:16:07,1,trendyol,Çorum,154.170,30.830,185.00,zübeyde DEMİREL,trendyolekspress,Onaylandı
92977,0888556689,002369,NaN,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,HepsiBurada,İzmir,33.325,6.665,39.99,İrem Erbey,hepsijet,Onaylandı
92978,2036956451,002284,yatak54896513739,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,trendyol,Konya,142.500,28.500,171.00,Esma Söylemez,trendyolekspress,Onaylandı


In [116]:
df_entegra_select_var["order_number"].nunique()

92290

##### Temp

In [113]:
df_entegra_select_duplicated = df_entegra_select_var[df_entegra_select_var.duplicated(keep=False)]
df_entegra_select_duplicated

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,total,tax,grand_total,company,cargo_company,status_name
1443,0408757643,003309,NaN,Champion Adalılar Kauçuk Sıcak Su Torbası Mavi...,2024-08-31 19:24:52,1,HepsiBurada,Konya,195.8333,39.1667,235.0000,Metin Karagöz,yurtici,Onaylandı
1444,0408757643,003309,NaN,Champion Adalılar Kauçuk Sıcak Su Torbası Mavi...,2024-08-31 19:24:52,1,HepsiBurada,Konya,195.8333,39.1667,235.0000,Metin Karagöz,yurtici,Onaylandı
1614,0407760580,002369,NaN,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-08-31 10:15:55,4,HepsiBurada,İstanbul,205.3552,36.2448,241.6000,Betül Peker,yurtici,Onaylandı
1615,0407760580,002369,NaN,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-08-31 10:15:55,4,HepsiBurada,İstanbul,205.3552,36.2448,241.6000,Betül Peker,yurtici,Onaylandı
1618,0194437964,002583,14421524554,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-08-31 09:57:09,1,HepsiBurada,Aydın,174.5455,17.4546,192.0001,Sedat Özkan,yurtici,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89815,0662726064,003570,NaN,Dekoratif Fişli 5 Metre Yıldızlı Pirinç Led Gü...,2024-03-10 07:23:13,1,HepsiBurada,Gaziantep,108.3250,21.6650,129.9900,Halime Özkılınç,hepsijet,Onaylandı
91849,0135890958,003876,NaN,Nem Alıcı Rutubet Ve Küf Önleyici Kare Aparat ...,2024-03-04 20:27:53,3,HepsiBurada,İstanbul,267.3750,53.4750,320.8500,Nilgün Dönmez,mng,Onaylandı
91850,0135890958,003876,NaN,Nem Alıcı Rutubet Ve Küf Önleyici Kare Aparat ...,2024-03-04 20:27:53,3,HepsiBurada,İstanbul,267.3750,53.4750,320.8500,Nilgün Dönmez,mng,Onaylandı
92810,0942304624,001603,NaN,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 16:24:32,1,HepsiBurada,İzmir,139.1583,27.8317,166.9900,esen ortaç,hepsijet,Onaylandı


In [115]:
# df_entegra_select_duplicated.to_excel(fr"{output_path}\Entegra Aynı Sipariş Tekrarlama ({variable_date}).xlsx", index=False)  # Hepsiburada siparişleri

In [28]:
# pd.to_datetime(df_entegra_select_var['datetime'], format='%Y-%m-%d %H:%M:%S')

In [125]:
df_entegra_select_var[df_entegra_select_var["order_number"] == '0237344754']

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,total,tax,grand_total,company,cargo_company,status_name
2095,0237344754,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-08-29 14:12:08,14,HepsiBurada,İstanbul,2443.6364,244.3636,2688.0,Yıldırım Topal,horoz,Onaylandı
2096,0237344754,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-08-29 14:12:08,7,HepsiBurada,İstanbul,1221.8182,122.1818,1344.0,Yıldırım Topal,horoz,Onaylandı
2097,0237344754,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-08-29 14:12:08,7,HepsiBurada,İstanbul,1221.8182,122.1818,1344.0,Yıldırım Topal,horoz,Onaylandı
2098,0237344754,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-08-29 14:12:08,7,HepsiBurada,İstanbul,1221.8182,122.1818,1344.0,Yıldırım Topal,mng,Onaylandı
2099,0237344754,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-08-29 14:12:08,7,HepsiBurada,İstanbul,1221.8182,122.1818,1344.0,Yıldırım Topal,mng,Onaylandı


In [126]:
df_entegra_select_var = df_entegra_select_var.sort_values(["datetime","total_product_quantity"], ascending=False)
df_entegra_select_var.reset_index(drop=True, inplace=True)
df_entegra_select_var

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,391.670,78.330,470.00,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,395.530,48.470,444.00,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,165.830,33.170,199.00,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,215.830,43.170,259.00,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,181.820,18.180,200.00,Damla Beşer,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92975,2036971893,001603,NaN,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,trendyol,Samsun,133.330,26.670,160.00,Gülizar Sarı,trendyolekspress,Onaylandı
92976,2036970386,002429,NaN,BTS3 Boyama Tuval Sayılı Set 25x35 Yunus,2024-03-01 00:16:07,1,trendyol,Çorum,154.170,30.830,185.00,zübeyde DEMİREL,trendyolekspress,Onaylandı
92977,0888556689,002369,NaN,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,HepsiBurada,İzmir,33.325,6.665,39.99,İrem Erbey,hepsijet,Onaylandı
92978,2036956451,002284,yatak54896513739,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,trendyol,Konya,142.500,28.500,171.00,Esma Söylemez,trendyolekspress,Onaylandı


In [127]:
df_entegra_select_var = df_entegra_select_var.drop_duplicates(subset="order_number", keep="first")
df_entegra_select_var = df_entegra_select_var.reset_index(drop=True)
df_entegra_select_var

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,391.670,78.330,470.00,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,395.530,48.470,444.00,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,165.830,33.170,199.00,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,215.830,43.170,259.00,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,181.820,18.180,200.00,Damla Beşer,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92285,2036971893,001603,NaN,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,trendyol,Samsun,133.330,26.670,160.00,Gülizar Sarı,trendyolekspress,Onaylandı
92286,2036970386,002429,NaN,BTS3 Boyama Tuval Sayılı Set 25x35 Yunus,2024-03-01 00:16:07,1,trendyol,Çorum,154.170,30.830,185.00,zübeyde DEMİREL,trendyolekspress,Onaylandı
92287,0888556689,002369,NaN,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,HepsiBurada,İzmir,33.325,6.665,39.99,İrem Erbey,hepsijet,Onaylandı
92288,2036956451,002284,yatak54896513739,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,trendyol,Konya,142.500,28.500,171.00,Esma Söylemez,trendyolekspress,Onaylandı


In [128]:
df_entegra_select_var[df_entegra_select_var["order_number"] == '0237344754']

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,total,tax,grand_total,company,cargo_company,status_name
2079,0237344754,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-08-29 14:12:08,14,HepsiBurada,İstanbul,2443.6364,244.3636,2688.0,Yıldırım Topal,horoz,Onaylandı


In [129]:
df_entegra_select_var.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92290 entries, 0 to 92289
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   order_number            92290 non-null  object        
 1   model                   92289 non-null  object        
 2   pov_productcode         28186 non-null  object        
 3   product_name            92289 non-null  object        
 4   datetime                92290 non-null  datetime64[ns]
 5   total_product_quantity  92290 non-null  int64         
 6   entegration             92290 non-null  object        
 7   invoice_city            92289 non-null  object        
 8   total                   92290 non-null  float64       
 9   tax                     92290 non-null  float64       
 10  grand_total             92290 non-null  float64       
 11  company                 92289 non-null  object        
 12  cargo_company           92290 non-null  object

In [130]:
df_entegra_select_var.loc[:,'datetime'] = pd.to_datetime(df_entegra_select_var.loc[:,'datetime'], format='%Y-%m-%d %H:%M:%S')
df_entegra_select_var.head(5)

,order_number,model,pov_productcode,product_name,datetime,total_product_quantity,entegration,invoice_city,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,391.67,78.33,470.0,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,395.53,48.47,444.0,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,165.83,33.17,199.0,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,215.83,43.17,259.0,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,181.82,18.18,200.0,Damla Beşer,trendyolekspress,Onaylandı


In [133]:
df_entegra_select_var.insert(3,"date",pd.to_datetime(df_entegra_select_var["datetime"]).dt.date)
df_entegra_select_var.insert(4,"time",pd.to_datetime(df_entegra_select_var["datetime"]).dt.time)
df_entegra_select_var.head(5)

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,invoice_city,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,NaN,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,trendyol,İstanbul,391.67,78.33,470.0,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,2024-09-05,10:50:42,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,trendyol,İstanbul,395.53,48.47,444.0,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,trendyol,Ankara,165.83,33.17,199.0,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,trendyol,İstanbul,215.83,43.17,259.0,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,trendyol,Sivas,181.82,18.18,200.0,Damla Beşer,trendyolekspress,Onaylandı


In [134]:
df_entegra_select_var.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92290 entries, 0 to 92289
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   order_number            92290 non-null  object        
 1   model                   92289 non-null  object        
 2   pov_productcode         28186 non-null  object        
 3   date                    92290 non-null  object        
 4   time                    92290 non-null  object        
 5   product_name            92289 non-null  object        
 6   datetime                92290 non-null  datetime64[ns]
 7   total_product_quantity  92290 non-null  int64         
 8   entegration             92290 non-null  object        
 9   invoice_city            92289 non-null  object        
 10  total                   92290 non-null  float64       
 11  tax                     92290 non-null  float64       
 12  grand_total             92290 non-null  float6

In [135]:
object_columns = df_entegra_select_var.select_dtypes(include='object')
object_columns

,order_number,model,pov_productcode,date,time,product_name,entegration,invoice_city,company,cargo_company,status_name
0,2383110310,003362,NaN,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",trendyol,İstanbul,ali özdemir,trendyolekspress,Onaylandı
1,2383102583,004382,NaN,2024-09-05,10:50:42,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,trendyol,İstanbul,Tuğba Sayar,trendyolekspress,Onaylandı
2,2383091113,001602,NaN,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",trendyol,Ankara,Tuba Koca,trendyolekspress,Onaylandı
3,2383085464,001602,NaN,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",trendyol,İstanbul,Doğa Özkan,trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",trendyol,Sivas,Damla Beşer,trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...
92285,2036971893,001603,NaN,2024-03-01,00:16:54,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",trendyol,Samsun,Gülizar Sarı,trendyolekspress,Onaylandı
92286,2036970386,002429,NaN,2024-03-01,00:16:07,BTS3 Boyama Tuval Sayılı Set 25x35 Yunus,trendyol,Çorum,zübeyde DEMİREL,trendyolekspress,Onaylandı
92287,0888556689,002369,NaN,2024-03-01,00:14:02,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,HepsiBurada,İzmir,İrem Erbey,hepsijet,Onaylandı
92288,2036956451,002284,yatak54896513739,2024-03-01,00:05:22,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,trendyol,Konya,Esma Söylemez,trendyolekspress,Onaylandı


In [136]:
object_columns.columns

Index(['order_number', 'model', 'pov_productcode', 'date', 'time',
       'product_name', 'entegration', 'invoice_city', 'company',
       'cargo_company', 'status_name'],
      dtype='object')

In [137]:
for column in object_columns.columns:
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: convert_one_character_letter(x))
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: whitespace_del(x))
    df_entegra_select_var.loc[:,f"{column}"] = df_entegra_select_var.loc[:,f"{column}"].apply(lambda x: title_func(x))
    
df_entegra_select_var

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,invoice_city,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,Nan,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,Trendyol,İstanbul,391.6700,78.3300,470.00,Ali Özdemir,Trendyolekspress,Onaylandı
1,2383102583,004382,Nan,2024-09-05,10:50:42,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,Trendyol,İstanbul,395.5300,48.4700,444.00,Tuğba Sayar,Trendyolekspress,Onaylandı
2,2383091113,001602,Nan,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,Trendyol,Ankara,165.8300,33.1700,199.00,Tuba Koca,Trendyolekspress,Onaylandı
3,2383085464,001602,Nan,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,Trendyol,İstanbul,215.8300,43.1700,259.00,Doğa Özkan,Trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,Trendyol,Sivas,181.8200,18.1800,200.00,Damla Beşer,Trendyolekspress,Onaylandı
5,2383074871,001602,Nan,2024-09-05,10:34:41,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:34:41,1,Trendyol,Denizli,165.8300,33.1700,199.00,Sare Hacer Var,Trendyolekspress,Onaylandı
6,2383069120,002643,290894121988,2024-09-05,10:30:18,Kaymaz Taban Tekli Post Peluş Banyo Paspası,2024-09-05 10:30:18,1,Trendyol,Bursa,90.0000,9.0000,99.00,Safinaz Dinç,Trendyolekspress,Onaylandı
7,2383066275,002263,Ayk8683372407015Çkmk,2024-09-05,10:29:26,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-09-05 10:29:26,1,Trendyol,Ankara,304.1700,60.8300,365.00,Şeyma Ödemi̇ş,Mng,Onaylandı
8,4575428431,003548,Nan,2024-09-05,10:28:49,"Yüzük Zikirmatik Karışık Renk, Yeni Nesil Ziki...",2024-09-05 10:28:49,2,Hepsiburada,Samsun,49.9833,9.9967,59.98,Zeynep Rana Karaağaç,Hepsijet,Onaylandı
9,2383064808,004159,Nan,2024-09-05,10:28:11,Spider Örümcek Oyun Çadırı Çift Yönlü Giriş Çı...,2024-09-05 10:28:11,1,Trendyol,Mersin,329.1700,65.8300,395.00,Senem Cekli,Trendyolekspress,Onaylandı


In [ ]:
df_entegra_select_product = df_entegra_select_var[["model","product_name"]]
df_entegra_select_product

In [ ]:
df_entegra_select_product = df_entegra_select_product.drop_duplicates()
df_entegra_select_product

##### Temp

In [42]:
df_entegra_select_var

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,2280224850,003362,Nan,2024-07-11,16:21:44,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:21:44,1,Trendyol,Yalova,Türkiye,383.33,76.67,460.0,Sevim Velioglu,Trendyolekspress,Onaylandı
1,2280223780,001602,Nan,2024-07-11,16:20:54,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:20:54,1,Trendyol,Bolu,Türkiye,154.17,30.83,185.0,Aslı Ünen,Trendyolekspress,Onaylandı
2,2280223782,003954,Nan,2024-07-11,16:20:05,Awox Barmix Çubuk Blender 1000 W,2024-07-11 16:20:05,1,Trendyol,Ağrı,Türkiye,298.33,59.67,358.0,Rahime Alpaslan,Trendyolekspress,Onaylandı
3,2280222421,001602,Nan,2024-07-11,16:19:57,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11 16:19:57,2,Trendyol,İstanbul,Türkiye,300.00,60.00,360.0,Sıla Erol,Trendyolekspress,Onaylandı
4,2280220893,003362,Nan,2024-07-11,16:19:42,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11 16:19:42,1,Trendyol,Samsun,Türkiye,383.33,76.67,460.0,Gülser Doğmuş,Trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94875,207423367318-1,002583,160620230178,2024-01-01,00:18:00,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:18:00,1,N11,Kütahya,Türkiye,153.64,15.36,169.0,Meltem Koca,Mng,Yeni Siparis
94876,1935764998,002979,Nan,2024-01-01,00:14:43,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01 00:14:43,3,Trendyol,Adıyaman,Türkiye,318.18,31.82,350.0,Sıla Dikel,Trendyolekspress,Onaylandı
94877,1935755636,003362,Nan,2024-01-01,00:02:03,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01 00:02:03,1,Trendyol,Aydın,Türkiye,312.50,62.50,375.0,Özlem Yıldız,Trendyolekspress,Onaylandı
94878,201757367316-1,002583,160620230178,2024-01-01,00:02:00,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01 00:02:00,1,N11,Kocaeli,Türkiye,153.64,15.36,169.0,Ekrem Çelik,Mng,Yeni Siparis


In [125]:
df_mikro_price = pd.read_excel(r"C:\Users\user\Downloads\Kopya Ürün Listesi Fiyat Bilgisi.xlsx")  # ????
df_mikro_price = df_mikro_price.rename(columns={"Ürün Adı":"product_name","Model Kodu":"model","Gelişi":"geliş_fiyat","KDV ORANI":"KDV Oranı"})
df_mikro_price

,product_name,Varyant,Marka,Renk,model,Stok Kodu,SKU,Barkod,Gelişi,KDV Oranı
0,"%100 Doğal At Kılı Selülit Fırçası, Doğal At K...",NaN,proclis,NaN,002972,002972,NaN,NaN,60,20.0
1,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",NaN,proclis,Ahşap,002975,002975,NaN,NaN,25,20.0
2,%100 Doğal Sarı Kantaron Yağı 50 ml,NaN,proclis,NaN,002960,002960,NaN,NaN,28.74,20.0
3,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...,NaN,Circle Toys,NaN,002932,002932,NaN,NaN,118,20.0
4,1 Adet Ayakkabı Hurcu Gri Çizgili,NaN,ALAS,Gri,002924,002924,NaN,NaN,156,10.0
...,...,...,...,...,...,...,...,...,...,...
1573,Şeffaf Kedi Ve Köpek Düz Sırt Taşıma Çantası Sarı,NaN,GOBYPET,Sarı,003865,003865,NaN,NaN,200,20.0
1574,Şeffaf Kedi ve Köpek Düz Sırt Taşıma Çantası 8 KG,NaN,NaN,NaN,003865,003865,NaN,NaN,200,20.0
1575,Şeffaf Tekerlekli Kedi Köpek Taşıma Valizi Çek...,NaN,Bagline,ŞEFFAF,BAG-21,f,NaN,NaN,NaN,NaN
1576,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm,NaN,Intex,Çok Renkli,004004,004004,NaN,NaN,145,20.0


In [126]:
mikro_model_set = set(df_mikro_price["model"])

In [127]:
mikro_model_list = list(mikro_model_set)

In [128]:
df_entegra_select_var[df_entegra_select_var["model"].isin(mikro_model_list)]

,model,pov_productcode,product_name,date,time,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:21:44,2024-07-11 16:21:44,1,Trendyol,Yalova,Türkiye,383.33,76.67,460.0,Sevim Velioglu,Trendyolekspress,Onaylandı
1,001602,Nan,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:20:54,2024-07-11 16:20:54,1,Trendyol,Bolu,Türkiye,154.17,30.83,185.0,Aslı Ünen,Trendyolekspress,Onaylandı
2,003954,Nan,Awox Barmix Çubuk Blender 1000 W,2024-07-11,16:20:05,2024-07-11 16:20:05,1,Trendyol,Ağrı,Türkiye,298.33,59.67,358.0,Rahime Alpaslan,Trendyolekspress,Onaylandı
3,001602,Nan,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-07-11,16:19:57,2024-07-11 16:19:57,2,Trendyol,İstanbul,Türkiye,300.00,60.00,360.0,Sıla Erol,Trendyolekspress,Onaylandı
4,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-07-11,16:19:42,2024-07-11 16:19:42,1,Trendyol,Samsun,Türkiye,383.33,76.67,460.0,Gülser Doğmuş,Trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94833,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01,00:18:00,2024-01-01 00:18:00,1,N11,Kütahya,Türkiye,153.64,15.36,169.0,Meltem Koca,Mng,Yeni Siparis
94834,002979,Nan,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,2024-01-01,00:14:43,2024-01-01 00:14:43,3,Trendyol,Adıyaman,Türkiye,318.18,31.82,350.0,Sıla Dikel,Trendyolekspress,Onaylandı
94835,003362,Nan,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-01-01,00:02:03,2024-01-01 00:02:03,1,Trendyol,Aydın,Türkiye,312.50,62.50,375.0,Özlem Yıldız,Trendyolekspress,Onaylandı
94836,002583,160620230178,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",2024-01-01,00:02:00,2024-01-01 00:02:00,1,N11,Kocaeli,Türkiye,153.64,15.36,169.0,Ekrem Çelik,Mng,Yeni Siparis


In [129]:
df_model_price_no = df_entegra_select_var[~df_entegra_select_var["model"].isin(mikro_model_list)]
df_model_price_no

,model,pov_productcode,product_name,date,time,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
5,004286,Nan,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...,2024-07-11,16:18:35,2024-07-11 16:18:35,1,Trendyol,Ankara,Türkiye,366.67,73.33,440.00,Fırat Aksel,Trendyolekspress,Onaylandı
16,002263,Ayk8683372407029Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,15:46:27,2024-07-11 15:46:27,1,Trendyol,İstanbul,Türkiye,280.00,56.00,336.00,Rabia Acar,Mng,Onaylandı
19,002263,Ayk8683372407029Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,15:38:38,2024-07-11 15:38:38,2,Trendyol,İstanbul,Türkiye,370.90,74.18,445.08,Aylin Uygun,Mng,Onaylandı
25,002263,Ayk8683372407029Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,15:23:45,2024-07-11 15:23:45,1,Trendyol,İstanbul,Türkiye,288.33,57.67,346.00,Mustafa Tercan,Mng,Onaylandı
28,002263,Ayk8683372407029Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,15:17:06,2024-07-11 15:17:06,1,Trendyol,İstanbul,Türkiye,288.33,57.67,346.00,Emre Yıldız,Mng,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94816,Tanımsız.,Nan,Tanımsız,2024-01-01,01:38:13,2024-01-01 01:38:13,1,Trendyol,Konya,Türkiye,677.50,135.50,813.00,Şahide Yeşilmen,Trendyolekspress,Onaylandı
94820,003265,Nan,Vipaş Hostel Kalın Borulu Otel Tipi Çamaşır Ku...,2024-01-01,01:22:17,2024-01-01 01:22:17,1,Trendyol,İstanbul,Türkiye,454.17,90.83,545.00,Sadi̇ye Eri̇mli̇ Dal,Trendyolekspress,Onaylandı
94823,Tanımsız.,Nan,Tanımsız,2024-01-01,01:16:37,2024-01-01 01:16:37,1,Trendyol,Mardin,Türkiye,137.50,27.50,165.00,Tolga Göktürk Fidan,Trendyolekspress,Onaylandı
94825,002062,Nan,Müller Benzi̇n Enjektör Temi̇zleyi̇ci̇ (Üstün ...,2024-01-01,01:03:00,2024-01-01 01:03:00,1,N11,Sakarya,Türkiye,109.78,21.96,131.74,Önder Irme,Mng,Yeni Siparis


In [132]:
df_model_price_no_drop = df_model_price_no.drop_duplicates(subset=['model','pov_productcode'])
df_model_price_no_drop

,model,pov_productcode,product_name,date,time,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
5,004286,Nan,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...,2024-07-11,16:18:35,2024-07-11 16:18:35,1,Trendyol,Ankara,Türkiye,366.6700,73.3300,440.00,Fırat Aksel,Trendyolekspress,Onaylandı
16,002263,Ayk8683372407029Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,15:46:27,2024-07-11 15:46:27,1,Trendyol,İstanbul,Türkiye,280.0000,56.0000,336.00,Rabia Acar,Mng,Onaylandı
52,002263,Ayk8683372407015Çkmk,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik...",2024-07-11,14:26:32,2024-07-11 14:26:32,1,Trendyol,Afyonkarahisar,Türkiye,304.1700,60.8300,365.00,Hasan Kürşad Yavuz,Mng,Onaylandı
135,001284,Nan,Alas Çok Amaçlı Dörtlü Çekmece Düzenleyici Org...,2024-07-11,11:51:52,2024-07-11 11:51:52,2,Trendyol,Adana,Türkiye,192.4200,38.4800,230.90,Hürü Kaplan,Trendyolekspress,Onaylandı
137,002998,Nan,Doğal El Yapımı Ardıç Katranlı Sabun Çiftli (M...,2024-07-11,11:50:03,2024-07-11 11:50:03,1,Trendyol,Ankara,Türkiye,99.9900,20.0000,119.99,Ömer Faruk Özkan,Trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94296,002374,Nan,Da69 Ahşap Resim Şovalesi 37 Cm,2024-01-02,20:08:22,2024-01-02 20:08:22,1,Trendyol,Ankara,Türkiye,99.9900,20.0000,119.99,Emek Elektri̇k End. A.Ş.,Trendyolekspress,Onaylandı
94600,002061,Nan,Müller Di̇zel Enjektör Temizleyi̇ci̇ (Üstün Al...,2024-01-01,21:45:00,2024-01-01 21:45:00,4,N11,Kayseri,Türkiye,422.8100,84.5600,507.37,Hakan Üzel,Mng,Yeni Siparis
94616,002792,Nan,21512-21010 Karter Tapasi Kisa Universal,2024-01-01,20:50:18,2024-01-01 20:50:18,1,Hepsiburada,İzmir,Türkiye,112.4917,22.4983,134.99,Kadir Inanç Polat,Hepsijet,Onaylandı
94729,002300,Ocakyakıtı25042052,"Osmanlı Ocak Yakıtı, Reşo Yakıt, Alfa Fire Man...",2024-01-01,15:16:06,2024-01-01 15:16:06,1,Hepsiburada,İstanbul,Türkiye,122.8750,24.5750,147.45,Mustafa Bora İmamoğlu,Hepsijet,Onaylandı


In [ ]:
df_model_price_no_drop.to_excel(fr"{output_path}\Entegra Ürün Fiyat Listesinde Olmayan.xlsx", index=False)

##### Entegra Select And Geo Location

In [61]:
df_city_location = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Geo Location\Master\Entegra City Location.xlsx")
df_city_location.drop("id", axis=1, inplace=True)
df_city_location

,city,country,invoice_city,latitude,longitude
0,Adana,Turkey,Adana,37.143800,35.498409
1,Adiyaman,Turkey,Adıyaman,37.763750,38.276348
2,Adiyaman,Turkey,Adiyaman,37.763750,38.276348
3,Afyonkarahisar,Turkey,Afyonkarahisar,38.685273,30.642741
4,Afyonkarahisar,Turkey,Afyon,38.685273,30.642741
...,...,...,...,...,...
262,Yanbu,Saudi Arabia,Yanbu,24.094893,38.070525
263,Az Zulfi,Saudi Arabia,Zulfi,26.311619,44.832951
264,Qairawan,Tunisia,Qairawan,35.673549,10.098636
265,Al Khodh,Oman,Al Khodh,23.565997,58.118169


In [62]:
# df_city_location[df_city_location["city"].duplicated(keep=False)]

In [63]:
df_city_location["city"].nunique()

195

In [64]:
df_entegra_select_var

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,invoice_city,invoice_country,total,tax,grand_total,company,cargo_company,status_name
0,2383110310,003362,Nan,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,Trendyol,İstanbul,Türkiye,391.670,78.330,470.00,Ali Özdemir,Trendyolekspress,Onaylandı
1,2383102583,004382,Nan,2024-09-05,10:50:42,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,Trendyol,İstanbul,Türkiye,395.530,48.470,444.00,Tuğba Sayar,Trendyolekspress,Onaylandı
2,2383091113,001602,Nan,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,Trendyol,Ankara,Türkiye,165.830,33.170,199.00,Tuba Koca,Trendyolekspress,Onaylandı
3,2383085464,001602,Nan,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,Trendyol,İstanbul,Türkiye,215.830,43.170,259.00,Doğa Özkan,Trendyolekspress,Onaylandı
4,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,Trendyol,Sivas,Türkiye,181.820,18.180,200.00,Damla Beşer,Trendyolekspress,Onaylandı
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92722,2036971893,001603,Nan,2024-03-01,00:16:54,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,Trendyol,Samsun,Türkiye,133.330,26.670,160.00,Gülizar Sarı,Trendyolekspress,Onaylandı
92723,2036970386,002429,Nan,2024-03-01,00:16:07,Bts3 Boyama Tuval Sayılı Set 25X35 Yunus,2024-03-01 00:16:07,1,Trendyol,Çorum,Türkiye,154.170,30.830,185.00,Zübeyde Demi̇rel,Trendyolekspress,Onaylandı
92724,0888556689,002369,Nan,2024-03-01,00:14:02,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,Hepsiburada,İzmir,Türkiye,33.325,6.665,39.99,İrem Erbey,Hepsijet,Onaylandı
92725,2036956451,002284,Yatak54896513739,2024-03-01,00:05:22,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,Trendyol,Konya,Türkiye,142.500,28.500,171.00,Esma Söylemez,Trendyolekspress,Onaylandı


In [65]:
df_entegra_city_location = pd.merge(df_entegra_select_var, df_city_location, how="left", on="invoice_city")
df_entegra_city_location = df_entegra_city_location.drop_duplicates().reset_index(drop=True)
df_entegra_city_location

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,invoice_city,...,total,tax,grand_total,company,cargo_company,status_name,city,country,latitude,longitude
0,2383110310,003362,Nan,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,Trendyol,İstanbul,...,391.670,78.330,470.00,Ali Özdemir,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
1,2383102583,004382,Nan,2024-09-05,10:50:42,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,Trendyol,İstanbul,...,395.530,48.470,444.00,Tuğba Sayar,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
2,2383091113,001602,Nan,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,Trendyol,Ankara,...,165.830,33.170,199.00,Tuba Koca,Trendyolekspress,Onaylandı,Ankara,Turkey,39.920776,32.854050
3,2383085464,001602,Nan,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,Trendyol,İstanbul,...,215.830,43.170,259.00,Doğa Özkan,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
4,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,Trendyol,Sivas,...,181.820,18.180,200.00,Damla Beşer,Trendyolekspress,Onaylandı,Sivas,Turkey,39.749973,37.014426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92722,2036971893,001603,Nan,2024-03-01,00:16:54,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,Trendyol,Samsun,...,133.330,26.670,160.00,Gülizar Sarı,Trendyolekspress,Onaylandı,Samsun,Turkey,41.294615,36.332060
92723,2036970386,002429,Nan,2024-03-01,00:16:07,Bts3 Boyama Tuval Sayılı Set 25X35 Yunus,2024-03-01 00:16:07,1,Trendyol,Çorum,...,154.170,30.830,185.00,Zübeyde Demi̇rel,Trendyolekspress,Onaylandı,Corum,Turkey,40.550470,34.954497
92724,0888556689,002369,Nan,2024-03-01,00:14:02,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,Hepsiburada,İzmir,...,33.325,6.665,39.99,İrem Erbey,Hepsijet,Onaylandı,Izmir,Turkey,38.423743,27.142802
92725,2036956451,002284,Yatak54896513739,2024-03-01,00:05:22,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,Trendyol,Konya,...,142.500,28.500,171.00,Esma Söylemez,Trendyolekspress,Onaylandı,Konya,Turkey,37.872734,32.492438


In [66]:
df_entegra_select = df_entegra_city_location.drop(["invoice_city","invoice_country"], axis=1)
df_entegra_select

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,total,tax,grand_total,company,cargo_company,status_name,city,country,latitude,longitude
0,2383110310,003362,Nan,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,Trendyol,391.670,78.330,470.00,Ali Özdemir,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
1,2383102583,004382,Nan,2024-09-05,10:50:42,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,Trendyol,395.530,48.470,444.00,Tuğba Sayar,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
2,2383091113,001602,Nan,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,Trendyol,165.830,33.170,199.00,Tuba Koca,Trendyolekspress,Onaylandı,Ankara,Turkey,39.920776,32.854050
3,2383085464,001602,Nan,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,Trendyol,215.830,43.170,259.00,Doğa Özkan,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
4,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,Trendyol,181.820,18.180,200.00,Damla Beşer,Trendyolekspress,Onaylandı,Sivas,Turkey,39.749973,37.014426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92722,2036971893,001603,Nan,2024-03-01,00:16:54,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,Trendyol,133.330,26.670,160.00,Gülizar Sarı,Trendyolekspress,Onaylandı,Samsun,Turkey,41.294615,36.332060
92723,2036970386,002429,Nan,2024-03-01,00:16:07,Bts3 Boyama Tuval Sayılı Set 25X35 Yunus,2024-03-01 00:16:07,1,Trendyol,154.170,30.830,185.00,Zübeyde Demi̇rel,Trendyolekspress,Onaylandı,Corum,Turkey,40.550470,34.954497
92724,0888556689,002369,Nan,2024-03-01,00:14:02,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,Hepsiburada,33.325,6.665,39.99,İrem Erbey,Hepsijet,Onaylandı,Izmir,Turkey,38.423743,27.142802
92725,2036956451,002284,Yatak54896513739,2024-03-01,00:05:22,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,Trendyol,142.500,28.500,171.00,Esma Söylemez,Trendyolekspress,Onaylandı,Konya,Turkey,37.872734,32.492438


In [67]:
df_entegra_select["latitude"] = df_entegra_select["latitude"].apply(lambda x: round(x, 6))
df_entegra_select["longitude"] = df_entegra_select["longitude"].apply(lambda x: round(x, 6))
df_entegra_select.head(5)

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,total,tax,grand_total,company,cargo_company,status_name,city,country,latitude,longitude
0,2383110310,003362,Nan,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,Trendyol,391.67,78.33,470.0,Ali Özdemir,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
1,2383102583,004382,Nan,2024-09-05,10:50:42,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,Trendyol,395.53,48.47,444.0,Tuğba Sayar,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
2,2383091113,001602,Nan,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,Trendyol,165.83,33.17,199.0,Tuba Koca,Trendyolekspress,Onaylandı,Ankara,Turkey,39.920776,32.854050
3,2383085464,001602,Nan,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,Trendyol,215.83,43.17,259.0,Doğa Özkan,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
4,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,Trendyol,181.82,18.18,200.0,Damla Beşer,Trendyolekspress,Onaylandı,Sivas,Turkey,39.749973,37.014426


In [68]:
print(df_entegra_select["product_name"].dtype)

object


In [69]:
print(df_entegra_select["latitude"].dtype)

float64


In [70]:
df_entegra_select["city"].nunique()

195

In [71]:
# df_entegra_select.to_excel(fr"{output_path}\Entegra Sales City Location For Tableau ({variable_date}).xlsx", index=False)  # Adoba..Data Folder

##### Brand Select For Tableau

In [72]:
df_entegra_select

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,total,tax,grand_total,company,cargo_company,status_name,city,country,latitude,longitude
0,2383110310,003362,Nan,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,Trendyol,391.670,78.330,470.00,Ali Özdemir,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
1,2383102583,004382,Nan,2024-09-05,10:50:42,Çok Amaçlı Multi İnce Temizlik Fırçası Derz Ar...,2024-09-05 10:50:42,7,Trendyol,395.530,48.470,444.00,Tuğba Sayar,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
2,2383091113,001602,Nan,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,Trendyol,165.830,33.170,199.00,Tuba Koca,Trendyolekspress,Onaylandı,Ankara,Turkey,39.920776,32.854050
3,2383085464,001602,Nan,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,Trendyol,215.830,43.170,259.00,Doğa Özkan,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872
4,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,Trendyol,181.820,18.180,200.00,Damla Beşer,Trendyolekspress,Onaylandı,Sivas,Turkey,39.749973,37.014426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92722,2036971893,001603,Nan,2024-03-01,00:16:54,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,Trendyol,133.330,26.670,160.00,Gülizar Sarı,Trendyolekspress,Onaylandı,Samsun,Turkey,41.294615,36.332060
92723,2036970386,002429,Nan,2024-03-01,00:16:07,Bts3 Boyama Tuval Sayılı Set 25X35 Yunus,2024-03-01 00:16:07,1,Trendyol,154.170,30.830,185.00,Zübeyde Demi̇rel,Trendyolekspress,Onaylandı,Corum,Turkey,40.550470,34.954497
92724,0888556689,002369,Nan,2024-03-01,00:14:02,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,Hepsiburada,33.325,6.665,39.99,İrem Erbey,Hepsijet,Onaylandı,Izmir,Turkey,38.423743,27.142802
92725,2036956451,002284,Yatak54896513739,2024-03-01,00:05:22,Kalorifer Petek Radyatör Yatağı Kedi Geniş Ked...,2024-03-01 00:05:22,1,Trendyol,142.500,28.500,171.00,Esma Söylemez,Trendyolekspress,Onaylandı,Konya,Turkey,37.872734,32.492438


In [73]:
df_brand_select = pd.read_excel(r"C:\Users\user\Desktop\Fiyat Bilgisi Olmayan\Entegra Ürün Satış Listesi Fiyat Barkod Düzenleme Master.xlsx", sheet_name="Sayfa2")
df_brand_select

,marka,model
0,Pikas,3415
1,Pikas,000028
2,Pikas,000037
3,Pikas,000863
4,Pikas,000864
...,...,...
444,Proclis,004390
445,Proclis,copy004229
446,Proclis,copy004230
447,Gobypet,copy004277


In [74]:
df_brand_tableau = pd.merge(df_entegra_select,df_brand_select, how='inner', on='model')
df_brand_tableau.drop_duplicates(inplace=True)
df_brand_tableau.dropna(inplace=True)
df_brand_tableau

,order_number,model,pov_productcode,date,time,product_name,datetime,total_product_quantity,entegration,total,tax,grand_total,company,cargo_company,status_name,city,country,latitude,longitude,marka
0,2383110310,003362,Nan,2024-09-05,10:54:27,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö...",2024-09-05 10:54:27,1,Trendyol,391.670,78.330,470.00,Ali Özdemir,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872,Pikas
1,2383091113,001602,Nan,2024-09-05,10:43:15,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:43:15,1,Trendyol,165.830,33.170,199.00,Tuba Koca,Trendyolekspress,Onaylandı,Ankara,Turkey,39.920776,32.854050,Gobypet
2,2383085464,001602,Nan,2024-09-05,10:40:26,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:40:26,2,Trendyol,215.830,43.170,259.00,Doğa Özkan,Trendyolekspress,Onaylandı,Istanbul,Turkey,41.006381,28.975872,Gobypet
3,2383076067,002583,220647971025,2024-09-05,10:34:55,"Portatif Katlanabilir Masa, Piknik Masası, Kam...",2024-09-05 10:34:55,1,Trendyol,181.820,18.180,200.00,Damla Beşer,Trendyolekspress,Onaylandı,Sivas,Turkey,39.749973,37.014426,Pikas
4,2383074871,001602,Nan,2024-09-05,10:34:41,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",2024-09-05 10:34:41,1,Trendyol,165.830,33.170,199.00,Sare Hacer Var,Trendyolekspress,Onaylandı,Denizli,Turkey,37.827589,29.238954,Gobypet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52882,2037012301,003856,Nan,2024-03-01,00:53:07,Çg42 Ahşap Çocuk Mutfak Seti,2024-03-01 00:53:07,1,Trendyol,1105.000,221.000,1326.00,Güldağ Yıldırım,Trendyolekspress,Onaylandı,Antalya,Turkey,36.896420,30.714880,Rozitoys
52883,2036985367,001603,Nan,2024-03-01,00:25:53,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:25:53,3,Trendyol,173.330,34.670,207.99,Sude Bacak,Mng,Onaylandı,Elazig,Turkey,38.582477,39.396179,Gobypet
52884,2036971893,001603,Nan,2024-03-01,00:16:54,"Pikas Ahşap Özel İsim Baskılı, Kare Kedi Köpek...",2024-03-01 00:16:54,1,Trendyol,133.330,26.670,160.00,Gülizar Sarı,Trendyolekspress,Onaylandı,Samsun,Turkey,41.294615,36.332060,Gobypet
52885,0888556689,002369,Nan,2024-03-01,00:14:02,Küçük Çelik Uçlu Kedi Köpek Evcil Hayvan Tırna...,2024-03-01 00:14:02,1,Hepsiburada,33.325,6.665,39.99,İrem Erbey,Hepsijet,Onaylandı,Izmir,Turkey,38.423743,27.142802,Pikas


In [75]:
df_brand_tableau.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52846 entries, 0 to 52886
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   order_number            52846 non-null  object        
 1   model                   52846 non-null  object        
 2   pov_productcode         52846 non-null  object        
 3   date                    52846 non-null  object        
 4   time                    52846 non-null  object        
 5   product_name            52846 non-null  object        
 6   datetime                52846 non-null  datetime64[ns]
 7   total_product_quantity  52846 non-null  int64         
 8   entegration             52846 non-null  object        
 9   total                   52846 non-null  float64       
 10  tax                     52846 non-null  float64       
 11  grand_total             52846 non-null  float64       
 12  company                 52846 non-null  object     

In [76]:
# df_brand_tableau.to_excel(fr"{output_path}\Entegra Sales Brand Select City Location For Tableau ({variable_date}).xlsx", index=False)  # Adoba..Data Folder

###### Stok Sayım

In [79]:
df_brand_select_sum = df_brand_tableau.groupby(["model","product_name","marka"])["total_product_quantity"].sum().reset_index()
df_brand_select_sum

,model,product_name,marka,total_product_quantity
0,000889,Pikas Suni Deri Çanta Sırt Askılı Fermuarlı Kı...,Pikas,2
1,000890,Pikas Suni Deri Çanta Sırt Askılı Fermuarlı Ye...,Pikas,1
2,001407,Pikas Vie Collection Su Geçirmez Yatak Koruyuc...,Pikas,5
3,001508,Pikas Portatif Katlanabilir Plastik Sehpa Antr...,Pikas,26
4,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",Gobypet,6191
...,...,...,...,...
361,004364,Çift Camlı Çift Cidarlı Cam Kupa Ev Ofis Finca...,Pikas,7
362,004371,6 Kişilik Taşınabilir Piknik Kamp Seti Taşıma ...,Pikas,41
363,004373,3 Lü Sızdırmaz Depolama Erzak Bakliyat Saklama...,Pikas,3
364,004380,"Kadın Kaş Kesme Makası, Taraklı Kaş Makası",Pikas,159


In [80]:
df_stok = pd.read_excel(r"C:\Users\user\Downloads\Stok Bilgisi.xlsx")
df_stok

,model,stok_miktar
0,000016,1.0
1,000053,20.0
2,000150,1.0
3,000479,44.0
4,000480,22.0
...,...,...
1994,ReşoYakıtı26042027,0.0
1995,ReşoYakıtı26042085,0.0
1996,TYCEUTXTBN169287194671452,0.0
1997,yatak54896513739,183.0


In [81]:
df_count = pd.read_excel(r"C:\Users\user\Downloads\Sayım Bilgisi.xlsx")
df_count

,model,sayım
0,002263,384
1,002263,104
2,002263,56
3,002263,280
4,002263,2
...,...,...
367,000922,"290,00"
368,004371,"10,00"
369,004233,"270,00"
370,001684,"40,00"


In [83]:
df1 = pd.merge(df_brand_select_sum, df_stok, how="left", on="model")
df1

,model,product_name,marka,total_product_quantity,stok_miktar
0,000889,Pikas Suni Deri Çanta Sırt Askılı Fermuarlı Kı...,Pikas,2,NaN
1,000890,Pikas Suni Deri Çanta Sırt Askılı Fermuarlı Ye...,Pikas,1,NaN
2,001407,Pikas Vie Collection Su Geçirmez Yatak Koruyuc...,Pikas,5,-43.0
3,001508,Pikas Portatif Katlanabilir Plastik Sehpa Antr...,Pikas,26,-43.0
4,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",Gobypet,6191,-7954.0
...,...,...,...,...,...
361,004364,Çift Camlı Çift Cidarlı Cam Kupa Ev Ofis Finca...,Pikas,7,94.0
362,004371,6 Kişilik Taşınabilir Piknik Kamp Seti Taşıma ...,Pikas,41,107.0
363,004373,3 Lü Sızdırmaz Depolama Erzak Bakliyat Saklama...,Pikas,3,NaN
364,004380,"Kadın Kaş Kesme Makası, Taraklı Kaş Makası",Pikas,159,794.0


In [84]:
df2 = pd.merge(df1, df_count, how="left", on="model")
df2

,model,product_name,marka,total_product_quantity,stok_miktar,sayım
0,000889,Pikas Suni Deri Çanta Sırt Askılı Fermuarlı Kı...,Pikas,2,NaN,NaN
1,000890,Pikas Suni Deri Çanta Sırt Askılı Fermuarlı Ye...,Pikas,1,NaN,NaN
2,001407,Pikas Vie Collection Su Geçirmez Yatak Koruyuc...,Pikas,5,-43.0,NaN
3,001508,Pikas Portatif Katlanabilir Plastik Sehpa Antr...,Pikas,26,-43.0,NaN
4,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ...",Gobypet,6191,-7954.0,NaN
...,...,...,...,...,...,...
399,004371,6 Kişilik Taşınabilir Piknik Kamp Seti Taşıma ...,Pikas,41,107.0,"93,00"
400,004371,6 Kişilik Taşınabilir Piknik Kamp Seti Taşıma ...,Pikas,41,107.0,"10,00"
401,004373,3 Lü Sızdırmaz Depolama Erzak Bakliyat Saklama...,Pikas,3,NaN,NaN
402,004380,"Kadın Kaş Kesme Makası, Taraklı Kaş Makası",Pikas,159,794.0,800


In [85]:
# df2.to_excel(fr"{output_path}\Entegra Sales Brand Stok Count (01.01.2024-11.07.2024).xlsx", index=False)

##### Sales Order Quantity

In [ ]:
df_entegra_select["status_name"].unique()

In [ ]:
entegration_list_all = list(df_entegra_select["entegration"].unique())
entegration_list_all

In [ ]:
df_entegra_select_sales = df_entegra_select[(df_entegra_select["status_name"] == "Onaylandı") | (df_entegra_select["status_name"] == "Yeni Siparis")]
df_entegra_select_sales.reset_index(drop=True, inplace=True)
df_entegra_select_sales

In [ ]:
entegration_list_sales = list(df_entegra_select_sales["entegration"].unique())
entegration_list_sales

In [ ]:
df_sales_quantity_entegration = pd.pivot_table(index="model",columns="entegration", values="total_product_quantity", aggfunc='sum', data=df_entegra_select_sales)
df_sales_quantity_entegration = df_sales_quantity_entegration.fillna(0)
df_sales_quantity_entegration.reset_index(inplace=True)
df_sales_quantity_entegration

In [ ]:
df_sales_quantity_entegration = df_sales_quantity_entegration.rename_axis(None, axis=1)  # del index name (entegration)
df_sales_quantity_entegration

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_quantity_entegration[f"{entegration}"] = 0
    else:
        pass
    
df_sales_quantity_entegration

In [ ]:
# 1 way
df_sales_quantity_entegration["toplam_satış_adet"] = 0
for entg_column in entegration_list_all:
    df_sales_quantity_entegration["toplam_satış_adet"] += df_sales_quantity_entegration[f"{entg_column}"]
# 2 way
# df_sales_quantity_entegration["total_product_sales"] = df_sales_quantity_entegration[entegration_list_all].sum(axis=1)

# 3 way manuel sum technique
# df_sales_quantity_entegration["total_product_sales"] = df_sales_quantity_entegration["Amazon"] + df_sales_quantity_entegration["Ciceksepeti"] + \
#     df_sales_quantity_entegration["Hepsiburada"] + df_sales_quantity_entegration["N11"] + df_sales_quantity_entegration["Trendyol"] + df_sales_quantity_entegration["Woocommerce"]
# df_sales_quantity_entegration

df_sales_quantity_entegration 

In [ ]:
df_sales_name_quantity_entegration  = pd.merge(df_sales_quantity_entegration, df_entegra_select_product, how="left", on="model")
df_sales_name_quantity_entegration.drop_duplicates(inplace=True)
df_sales_name_quantity_entegration

In [ ]:
cols = list(df_sales_name_quantity_entegration.columns)
cols = [cols[-1]] + cols[:-1]
df_sales_name_quantity_entegration = df_sales_name_quantity_entegration[cols]
df_sales_name_quantity_entegration

In [ ]:
df_sales_name_quantity_entegration = df_sales_name_quantity_entegration.sort_values(by="toplam_satış_adet", ascending=False)
df_sales_name_quantity_entegration.reset_index(drop=True, inplace=True)
df_sales_name_quantity_entegration

In [ ]:
df_sales_name_quantity_entegration_drop = df_sales_name_quantity_entegration.drop_duplicates(subset="model")
df_sales_name_quantity_entegration_drop.reset_index(drop=True, inplace=True)
df_sales_name_quantity_entegration_drop

In [ ]:
df_sales_name_quantity_entegration_drop.to_excel(fr"{output_path}\Entegra Ürün Satış Rakamları Adet ({variable_date}).xlsx", index=False)

##### Sales Order TL

In [ ]:
df_entegra_select_sales

In [ ]:
df_entegration_model_sales = df_entegra_select_sales.groupby(["model","entegration"])[["total","tax","grand_total"]].sum()
df_entegration_model_sales = df_entegration_model_sales.reset_index()
df_entegration_model_sales.sort_values(by=["entegration","total"], ascending=False, inplace=True)
df_entegration_model_sales.reset_index(drop=True, inplace=True)
df_entegration_model_sales

###### Total

In [ ]:
df_sales_entegration_total = pd.pivot_table(index="model",columns="entegration", values="total", aggfunc='sum', data=df_entegration_model_sales)
df_sales_entegration_total = df_sales_entegration_total.fillna(0)
df_sales_entegration_total.reset_index(inplace=True)
df_sales_entegration_total = df_sales_entegration_total.rename_axis(None, axis=1)
df_sales_entegration_total

In [ ]:
# df_sales_entegration_total.isnull()

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_entegration_total[f"{entegration}"] = 0
    else:
        pass
    
df_sales_entegration_total

In [ ]:
# 1 way
df_sales_entegration_total["toplam_net_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_sales_entegration_total["toplam_net_satış_tl"] += df_sales_entegration_total[f"{entg_column}"]

df_sales_entegration_total 

In [ ]:
df_sales_entegration_total = df_sales_entegration_total.sort_values(by="toplam_net_satış_tl", ascending=False)
df_sales_entegration_total.reset_index(drop=True, inplace=True)
df_sales_entegration_total = df_sales_entegration_total[["model","toplam_net_satış_tl"]]
df_sales_entegration_total

###### Tax

In [ ]:
df_sales_entegration_tax = pd.pivot_table(index="model",columns="entegration", values="tax", aggfunc='sum', data=df_entegration_model_sales)
df_sales_entegration_tax = df_sales_entegration_tax.fillna(0)
df_sales_entegration_tax.reset_index(inplace=True)
df_sales_entegration_tax = df_sales_entegration_tax.rename_axis(None, axis=1)
df_sales_entegration_tax

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_entegration_tax[f"{entegration}"] = 0
    else:
        pass
    
df_sales_entegration_tax

In [ ]:
# 1 way
df_sales_entegration_tax["toplam_vergi_tl"] = 0
for entg_column in entegration_list_all:
    df_sales_entegration_tax["toplam_vergi_tl"] += df_sales_entegration_tax[f"{entg_column}"]

df_sales_entegration_tax

In [ ]:
df_sales_entegration_tax = df_sales_entegration_tax.sort_values(by="toplam_vergi_tl", ascending=False)
df_sales_entegration_tax.reset_index(drop=True, inplace=True)
df_sales_entegration_tax = df_sales_entegration_tax[["model","toplam_vergi_tl"]]
df_sales_entegration_tax

###### Grand Total

In [ ]:
df_sales_entegration_grand_total = pd.pivot_table(index="model",columns="entegration", values="grand_total", aggfunc='sum', data=df_entegration_model_sales)
df_sales_entegration_grand_total = df_sales_entegration_grand_total.fillna(0)
df_sales_entegration_grand_total.reset_index(inplace=True)
df_sales_entegration_grand_total = df_sales_entegration_grand_total.rename_axis(None, axis=1)
df_sales_entegration_grand_total

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_sales:
        df_sales_entegration_grand_total[f"{entegration}"] = 0
    else:
        pass
    
df_sales_entegration_grand_total

In [ ]:
# 1 way
df_sales_entegration_grand_total["toplam_brüt_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_sales_entegration_grand_total["toplam_brüt_satış_tl"] += df_sales_entegration_grand_total[f"{entg_column}"]

df_sales_entegration_grand_total

In [ ]:
df_sales_entegration_grand_total = df_sales_entegration_grand_total.sort_values(by="toplam_brüt_satış_tl", ascending=False)
df_sales_entegration_grand_total.reset_index(drop=True, inplace=True)
df_sales_entegration_grand_total = df_sales_entegration_grand_total[["model","toplam_brüt_satış_tl"]]
df_sales_entegration_grand_total

In [ ]:
data_sales_frames = [df_sales_entegration_total, df_sales_entegration_tax, df_sales_entegration_grand_total]

In [ ]:
df_sales_merged = reduce(lambda  left,right: pd.merge(left,right,on=['model'], how='inner'), data_sales_frames)
df_sales_merged = df_sales_merged.drop_duplicates()
df_sales_merged = df_sales_merged.reset_index(drop=True)
df_sales_merged

In [ ]:
df_quantity_sales_entegration_merge  = pd.merge(df_sales_name_quantity_entegration_drop, df_sales_merged, how="left", on="model")
df_quantity_sales_entegration_merge.drop_duplicates(inplace=True)
df_quantity_sales_entegration_merge

In [ ]:
df_quantity_sales_entegration_merge.to_excel(fr"{output_path}\Entegra Ürün Satış Adet Ve TL ({variable_date}).xlsx", index=False)

###### Sales Duplicated

In [ ]:
df_sales_dublicated_entegration = df_sales_name_quantity_entegration[df_sales_name_quantity_entegration.duplicated(subset="model", keep=False)]
df_sales_dublicated_entegration

In [ ]:
df_sales_dublicated_entegration.to_excel(fr"{output_path}\Entegra Ürün Satış Rakamları Adet Yenile.xlsx", index=False)

##### Sales Visualization

In [ ]:
df_sales_visual = df_sales_name_quantity_entegration_drop.head(50)
df_sales_visual

In [ ]:
select_sales_visual_list = list(df_sales_visual["model"])
# select_sales_visual_list

In [ ]:
# df_sales_name_quantity_entegration_drop.drop(columns="model", axis=1)

In [ ]:
#dfm = pd.melt(frame=df_sales_name_quantity_entegration_drop, id_vars=["product_name","model"],var_name='cols', value_name='vals')
dfm = df_sales_name_quantity_entegration_drop.melt(id_vars=["product_name","model"],var_name='cols', value_name='vals')
dfm

In [ ]:
dfm_select = dfm[dfm["model"].isin(select_sales_visual_list)]
dfm_select

In [ ]:
plt.figure(figsize=(16,9),dpi=100)
graph = sns.catplot(x="model", y="vals", hue='cols', data=dfm_select, kind='point', legend_out=False, height=10, aspect=2, linewidth=1.5)
# sns.pointplot(x="model", y="vals", hue='cols', data=dfm_select)
plt.xlabel("Model No")
plt.xticks(rotation=45)
plt.ylabel("Adet")
plt.title("Model Entegration Satış Rakamları")
plt.legend(title='Entegration', loc='upper right');

In [ ]:
df_sales_select_top = df_sales_name_quantity_entegration_drop.head(20)
df_sales_select_top

In [ ]:
df_sales_select_top.loc[:,"product_name"] + " | " + df_sales_select_top.loc[:,"model"]

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_sales_select_top, x="toplam_satış_adet", y="product_name", color="brown")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Adet")
# plt.xticks(rotation=45)

# ax.set_yticklabels(df_sales_select_top.loc[:,"product_name"] + " " + df_sales_select_top.loc[:,"model"])
ax.set_yticks(range(len(df_sales_select_top)))  # Etiketlerin konumlarını ayarla
ax.set_yticklabels(list(df_sales_select_top.loc[:,"model"] + " | " + df_sales_select_top.loc[:,"product_name"]))

plt.ylabel("Ürün")
plt.title("Ürün Toplam Satış Top 20")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

# # for value on graph 2 way
# i=0
# for p in graph.patches:
#     height = p.get_height()
#     graph.text(p.get_x()+p.get_width()/2., height + 0.5,
#         df_seller_evaluation['olumsuz etkiler'].value_counts()[i],ha="center")
#     i += 1;

In [ ]:
df_sales_select_last = df_sales_name_quantity_entegration_drop.tail(20)
df_sales_select_last

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_sales_select_last, x="toplam_satış_adet", y="product_name", color="brown")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Adet")
# plt.xticks(rotation=45)

ax.set_yticks(range(len(df_sales_select_last)))  # Etiketlerin konumlarını ayarla
ax.set_yticklabels(list(df_sales_select_last.loc[:,"model"] + " | " + df_sales_select_last.loc[:,"product_name"]))

plt.ylabel("Ürün")
plt.title("Ürün Toplam Satış Last 20")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

##### Entegration Sales Number

In [ ]:
dfm

In [ ]:
dfm2 = dfm[dfm["cols"].isin(entegration_list_all)]
dfm2 = dfm2.dropna()
dfm2 = dfm2[dfm2["vals"] != 0]
dfm2.reset_index(drop=True, inplace=True)
dfm2

In [ ]:
plt.figure(figsize=(16,9),dpi=100)
graph = sns.countplot(x="cols", data=dfm2, color="green")
plt.xlabel("Entegration")
plt.xticks(rotation=45)
plt.ylabel("Adet")
plt.title("Entegration Tekil Ürün Satış")
# 1 way
# graph.bar_label(graph.containers[0])
# 2 way
for container in graph.containers:
    graph.bar_label(container)

# Note: Grafik pazaryerlerinde satılan ürün çeşidi sayısını göstermekte.

In [ ]:
dfm3 = dfm2.groupby(["cols"])[["vals"]].sum().reset_index()
dfm3

In [ ]:
dfm3.idxmax()

In [ ]:
dfm3.index.max()

In [ ]:
dfm3.loc[dfm3.index.max()+1,:] = {"cols":"Total", "vals":dfm3["vals"].sum()}  # Total sales quantity
dfm3

In [ ]:
# i = 0
# for value in values:
#     df.loc[df.index.max()+i,:] = {"cols":"Total", "vals":value}
#     i += 1 

In [ ]:
plt.figure(figsize=(16,9),dpi=100)
graph = sns.barplot(x="cols", y="vals", data=dfm3, color="orange",)
plt.xlabel("Entegration")
plt.xticks(rotation=45)
plt.ylabel("Adet")
plt.title("Entegration Toplam Satış")
# 1 way
graph.bar_label(graph.containers[0]);

# Note: Grafik pazaryerlerinde ve toplamda satılan toplam ürün sayısını (bir şipariş birden fazla ürün içerebileceğinden ürün sayısı olarak ele alındı) göstermekte.

##### Cancel Order Quantity

In [ ]:
df_entegra_select["status_name"].unique()

In [ ]:
df_entegra_select_cancel = df_entegra_select[(df_entegra_select["status_name"] == "İade-İptal")]
df_entegra_select_cancel.reset_index(drop=True, inplace=True)
df_entegra_select_cancel

In [ ]:
entegration_list_all = list(df_entegra_select["entegration"].unique())
entegration_list_all

In [ ]:
entegration_list_cancel = list(df_entegra_select_cancel["entegration"].unique())
entegration_list_cancel

In [ ]:
df_cancel_quantity_entegration = pd.pivot_table(index="model",columns="entegration", values="total_product_quantity", aggfunc='sum', data=df_entegra_select_cancel)
df_cancel_quantity_entegration = df_cancel_quantity_entegration.fillna(0)
df_cancel_quantity_entegration.reset_index(inplace=True)
df_cancel_quantity_entegration

In [ ]:
df_cancel_quantity_entegration = df_cancel_quantity_entegration.rename_axis(None, axis=1)  # del index name
df_cancel_quantity_entegration

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_quantity_entegration[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_quantity_entegration

In [ ]:
# 1 way
# df_cancel_quantity_entegration["total_product_cancel"] = 0
# for entg_column in entegration_list_all:
#     df_cancel_quantity_entegration["total_product_cancel"] += df_cancel_quantity_entegration[f"{entg_column}"]

# 2 way
df_cancel_quantity_entegration["toplam_iptal_adet"] = df_cancel_quantity_entegration[entegration_list_all].sum(axis=1)

df_cancel_quantity_entegration

In [ ]:
# df_cancel_quantity_entegration["total_product_cancel"] = df_cancel_quantity_entegration["Amazon"] + df_cancel_quantity_entegration["Ciceksepeti"] + \
#     df_cancel_quantity_entegration["Hepsiburada"] + df_cancel_quantity_entegration["N11"] + df_cancel_quantity_entegration["Trendyol"] + df_cancel_quantity_entegration["Woocommerce"]
# df_cancel_quantity_entegration

In [ ]:
df_cancel_name_quantity_entegration  = pd.merge(df_cancel_quantity_entegration, df_entegra_select_product, how="left", on="model")
df_cancel_name_quantity_entegration.drop_duplicates(inplace=True)
df_cancel_name_quantity_entegration

In [ ]:
# df_cancel_name_quantity_entegration[[df_cancel_name_quantity_entegration.columns[-1]]]

In [ ]:
cols = list(df_cancel_name_quantity_entegration.columns)
cols = [cols[-1]] + cols[:-1]
df_cancel_name_quantity_entegration = df_cancel_name_quantity_entegration[cols]
df_cancel_name_quantity_entegration

In [ ]:
df_cancel_name_quantity_entegration = df_cancel_name_quantity_entegration.sort_values(by="toplam_iptal_adet", ascending=False)
df_cancel_name_quantity_entegration.reset_index(drop=True, inplace=True)
df_cancel_name_quantity_entegration

In [ ]:
df_cancel_name_quantity_entegration_drop = df_cancel_name_quantity_entegration.drop_duplicates(subset="model")
df_cancel_name_quantity_entegration_drop.reset_index(drop=True, inplace=True)
df_cancel_name_quantity_entegration_drop

In [ ]:
df_cancel_name_quantity_entegration_drop.to_excel(fr"{output_path}\Entegra Ürün İptal Rakamları Adet.xlsx", index=False)

##### Cancel Order TL

In [ ]:
df_entegra_select_cancel

In [ ]:
df_entegration_model_cancel = df_entegra_select_cancel.groupby(["model","entegration"])[["total","tax","grand_total"]].sum()
df_entegration_model_cancel = df_entegration_model_cancel.reset_index()
df_entegration_model_cancel.sort_values(by=["entegration","total"], ascending=False, inplace=True)
df_entegration_model_cancel.reset_index(drop=True, inplace=True)
df_entegration_model_cancel

###### İptal Satış

In [ ]:
df_cancel_entegration_total = pd.pivot_table(index="model",columns="entegration", values="total", aggfunc='sum', data=df_entegration_model_cancel)
df_cancel_entegration_total = df_cancel_entegration_total.fillna(0)
df_cancel_entegration_total.reset_index(inplace=True)
df_cancel_entegration_total = df_cancel_entegration_total.rename_axis(None, axis=1)
df_cancel_entegration_total

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_entegration_total[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_entegration_total

In [ ]:
# 1 way
df_cancel_entegration_total["toplam_iptal_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_cancel_entegration_total["toplam_iptal_satış_tl"] += df_cancel_entegration_total[f"{entg_column}"]

df_cancel_entegration_total 

In [ ]:
df_cancel_entegration_total = df_cancel_entegration_total.sort_values(by="toplam_iptal_satış_tl", ascending=False)
df_cancel_entegration_total.reset_index(drop=True, inplace=True)
df_cancel_entegration_total = df_cancel_entegration_total[["model","toplam_iptal_satış_tl"]]
df_cancel_entegration_total

###### İptal Vergi

In [ ]:
df_cancel_entegration_tax = pd.pivot_table(index="model",columns="entegration", values="tax", aggfunc='sum', data=df_entegration_model_cancel)
df_cancel_entegration_tax = df_cancel_entegration_tax.fillna(0)
df_cancel_entegration_tax.reset_index(inplace=True)
df_cancel_entegration_tax = df_cancel_entegration_tax.rename_axis(None, axis=1)
df_cancel_entegration_tax

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_entegration_tax[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_entegration_tax

In [ ]:
# 1 way
df_cancel_entegration_tax["toplam_iptal_vergi_tl"] = 0
for entg_column in entegration_list_all:
    df_cancel_entegration_tax["toplam_iptal_vergi_tl"] += df_cancel_entegration_tax[f"{entg_column}"]

df_cancel_entegration_tax

In [ ]:
df_cancel_entegration_tax = df_cancel_entegration_tax.sort_values(by="toplam_iptal_vergi_tl", ascending=False)
df_cancel_entegration_tax.reset_index(drop=True, inplace=True)
df_cancel_entegration_tax = df_cancel_entegration_tax[["model","toplam_iptal_vergi_tl"]]
df_cancel_entegration_tax

###### Brüt İptal

In [ ]:
df_cancel_entegration_grand_total = pd.pivot_table(index="model",columns="entegration", values="grand_total", aggfunc='sum', data=df_entegration_model_cancel)
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total.fillna(0)
df_cancel_entegration_grand_total.reset_index(inplace=True)
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total.rename_axis(None, axis=1)
df_cancel_entegration_grand_total

In [ ]:
for entegration in entegration_list_all:
    if entegration not in entegration_list_cancel:
        df_cancel_entegration_grand_total[f"{entegration}"] = 0
    else:
        pass
    
df_cancel_entegration_grand_total

In [ ]:
# 1 way
df_cancel_entegration_grand_total["toplam_iptal_brüt_satış_tl"] = 0
for entg_column in entegration_list_all:
    df_cancel_entegration_grand_total["toplam_iptal_brüt_satış_tl"] += df_cancel_entegration_grand_total[f"{entg_column}"]

df_cancel_entegration_grand_total

In [ ]:
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total.sort_values(by="toplam_iptal_brüt_satış_tl", ascending=False)
df_cancel_entegration_grand_total.reset_index(drop=True, inplace=True)
df_cancel_entegration_grand_total = df_cancel_entegration_grand_total[["model","toplam_iptal_brüt_satış_tl"]]
df_cancel_entegration_grand_total

In [ ]:
data_cancel_frames = [df_cancel_entegration_total, df_cancel_entegration_tax, df_cancel_entegration_grand_total]

In [ ]:
df_cancel_merged = reduce(lambda  left,right: pd.merge(left,right,on=['model'], how='inner'), data_cancel_frames)
df_cancel_merged = df_cancel_merged.drop_duplicates()
df_cancel_merged = df_cancel_merged.reset_index(drop=True)
df_cancel_merged

In [ ]:
df_quantity_cancel_entegration_merge  = pd.merge(df_cancel_name_quantity_entegration_drop, df_cancel_merged, how="left", on="model")
df_quantity_cancel_entegration_merge.drop_duplicates(inplace=True)
df_quantity_cancel_entegration_merge

In [ ]:
df_quantity_cancel_entegration_merge.to_excel(fr"{output_path}\Entegra Ürün İptal Satış Adet Ve TL.xlsx", index=False)

###### Cancel Duplicated

In [ ]:
df_cancel_dublicated_entegration = df_cancel_name_quantity_entegration[df_cancel_name_quantity_entegration.duplicated(subset="model", keep=False)]
df_cancel_dublicated_entegration

In [ ]:
df_cancel_dublicated_entegration.to_excel(fr"{output_path}\Entegra Ürün İptal Rakamları Adet Yenile.xlsx", index=False)

##### Cancel Visualization

In [ ]:
df_cancel_visual = df_cancel_name_quantity_entegration_drop.head(50)
df_cancel_visual

In [ ]:
select_cancel_visual_list = list(df_cancel_visual["model"])
# select_cancel_visual_list

In [ ]:
dfm2 = df_cancel_name_quantity_entegration_drop.melt(id_vars=["product_name","model"],var_name='cols', value_name='vals')
dfm2

In [ ]:
dfm_select2 = dfm2[dfm2["model"].isin(select_cancel_visual_list)]
dfm_select2

In [ ]:
plt.figure(figsize=(16,9),dpi=100)
graph = sns.catplot(x="model", y="vals", hue='cols', data=dfm_select2, kind='point', legend_out=False, height=10, aspect=2, linewidth=1.5)
# sns.pointplot(x="model", y="vals", hue='cols', data=dfm_select)
plt.xlabel("Model No")
plt.xticks(rotation=45)
plt.ylabel("Adet")
plt.title("Model İptal Rakamları")
plt.legend(title='Entegration', loc='upper right');

In [ ]:
df_cancel_select_top = df_cancel_name_quantity_entegration_drop.head(20)
df_cancel_select_top

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_cancel_select_top, x="toplam_iptal_adet", y="product_name", color="red")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Adet")
# plt.xticks(rotation=45)

# y eksenindeki etiketlere ekleme yapıyoruz. 
ax.set_yticks(range(len(df_cancel_select_top)))  # Etiketlerin konumlarını ayarla
ax.set_yticklabels(list(df_cancel_select_top.loc[:,"model"] + " | " + df_cancel_select_top.loc[:,"product_name"]))

plt.ylabel("Ürün")
plt.title("Ürün Toplam İptal Top 20")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

In [ ]:
df_cancel_select_last = df_cancel_name_quantity_entegration_drop.tail(20)
df_cancel_select_last

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_cancel_select_last, x="toplam_iptal_adet", y="product_name", color="red")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Adet")
# plt.xticks(rotation=45)
# y eksenindeki etiketlere ekleme yapıyoruz. 
ax.set_yticks(range(len(df_cancel_select_last)))  # Etiketlerin konumlarını ayarla
ax.set_yticklabels(list(df_cancel_select_last.loc[:,"model"] + " | " + df_cancel_select_last.loc[:,"product_name"]))

plt.ylabel("Ürün")
plt.title("Ürün Toplam Satış Last 20")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

##### Add Cancel Order To Sales

###### Part 1

In [ ]:
df_sales_name_quantity_entegration_drop

In [ ]:
df_cancel_name_quantity_entegration_drop_var = df_cancel_name_quantity_entegration_drop[["model","toplam_iptal_adet"]]
df_cancel_name_quantity_entegration_drop_var

In [ ]:
df_sales_cancel_quantity_all = pd.merge(df_sales_name_quantity_entegration_drop, df_cancel_name_quantity_entegration_drop_var, how="left", on="model")
df_sales_cancel_quantity_all.fillna(0, inplace=True)
df_sales_cancel_quantity_all

In [ ]:
df_sales_cancel_quantity_all.to_excel(fr"{output_path}\Entegra Ürün Satış Ve İptal Rakamları Adet Master.xlsx", index=False)

###### Part 2

In [ ]:
df_sales_merged

In [ ]:
df_cancel_merged

In [ ]:
df_sales_cancel_quantity_tl_all = pd.merge(df_sales_cancel_quantity_all, df_sales_merged, how="left", on="model")
df_sales_cancel_quantity_tl_all2 = pd.merge(df_sales_cancel_quantity_tl_all, df_cancel_merged, how="left", on="model")
df_sales_cancel_quantity_tl_all2.fillna(0, inplace=True)
df_sales_cancel_quantity_tl_all2

In [ ]:
df_sales_cancel_quantity_tl_all2.to_excel(fr"{output_path}\Entegra Ürün Satış Ve İptal Adet Ve TL Master.xlsx", index=False)

##### Sales Cancel Visualization

In [ ]:
df_sales_cancel_all_select = df_sales_cancel_quantity_all.head(30)
df_sales_cancel_all_select

In [ ]:
fig = plt.figure(figsize=(16,9), dpi=100)
ax = fig.add_axes([0,0,1,1])
ax.plot(df_sales_cancel_all_select["model"],df_sales_cancel_all_select["toplam_satış_adet"],label='Ürün Satış')
ax.plot(df_sales_cancel_all_select["model"],df_sales_cancel_all_select["toplam_iptal_adet"],label="Ürün İptal")
ax.set_title("Ürün Satış Ve İptal")
ax.set_ylabel("Miktar Adet")
ax.set_xlabel("Ürün")
plt.xticks(rotation=45)
#ax.set_xticklabels(ax.get_xticks(), rotation = 40)
ax.legend(loc=0);

In [ ]:
df_sales_total_tax = pd.DataFrame(df_entegra_select_sales[["total","tax","grand_total"]].sum()).reset_index()
df_sales_total_tax.rename(columns={"index":"name", 0:"value"}, inplace=True)
df_sales_total_tax

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_sales_total_tax, x="name", y="value", color="green")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Ad")
# plt.xticks(rotation=45)

# x eksenindeki etiketleri büyük harf yapalım
xticklabel_var = [label.get_text().capitalize() for label in graph.get_xticklabels()]
ax.set_xticks(range(len(xticklabel_var))) # Etiketlerin konumlarını ayarla
ax.set_xticklabels(xticklabel_var)


plt.ylabel("Miktar TL")
plt.title("Grand Total-Tax-Sales")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

In [ ]:
df_cancel_total_tax = pd.DataFrame(df_entegra_select_cancel[["total","tax","grand_total"]].sum()).reset_index()
df_cancel_total_tax.rename(columns={"index":"name", 0:"value"}, inplace=True)
df_cancel_total_tax

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)
graph = sns.barplot(data=df_cancel_total_tax, x="name", y="value", color="red")
# ax.set_xticks(np.arange(0, 101, 5))
plt.xlabel("Ad")
# plt.xticks(rotation=45)

# x eksenindeki etiketleri büyük harf yapalım
xticklabel_var = [label.get_text().capitalize() for label in graph.get_xticklabels()]
ax.set_xticks(range(len(xticklabel_var))) # Etiketlerin konumlarını ayarla
ax.set_xticklabels(xticklabel_var)


plt.ylabel("Miktar TL")
plt.title("Grand Total-Tax-Cancel")

# for value on graph 1 way
graph.bar_label(ax.containers[0]);

In [ ]:
# for i in graph.get_xticklabels():
#     print(i.get_text())

In [ ]:
# for i in [label.get_text() for label in graph.get_xticklabels()]:
#     print(str.capitalize(i))

In [ ]:
df_entegra_total_tax = df_entegra_select[["entegration","total","tax","grand_total"]]
df_entegra_total_tax

In [ ]:
df_grouped = df_entegra_total_tax.groupby('entegration').sum().reset_index()

# Melt işlemi ile total, tax, grand_total sütunlarını birleştirelim
df_melted = pd.melt(df_grouped, id_vars=['entegration'], value_vars=['total', 'tax', 'grand_total'], var_name='value_type', value_name='value')

# Grafik çizimi için seaborn barplot kullanalım
plt.figure(figsize=(16, 9))
graph = sns.barplot(data=df_melted, x='entegration', y='value', hue='value_type', palette='bright')

# Grafik başlığı ve eksen etiketlerini ekleyelim
plt.title('Toplam, Vergi Ve Grand Total')
plt.xlabel('Entegrasyon')
plt.ylabel('Miktar TL')

# Görseli gösterelim
plt.tight_layout()
plt.show()

In [ ]:
df_grouped = df_entegra_total_tax.groupby('entegration').sum().reset_index()

# Melt işlemi ile total, tax, grand_total sütunlarını birleştirelim
df_melted = pd.melt(df_grouped, id_vars=['entegration'], value_vars=['total', 'tax', 'grand_total'], var_name='value_type', value_name='value')

# Grafik çizimi için seaborn barplot kullanalım
plt.figure(figsize=(16, 9))
graph = sns.barplot(data=df_melted, x='value_type', y='value', hue='entegration', palette='bright')

# Grafik başlığı ve eksen etiketlerini ekleyelim
plt.title('Toplam, Vergi Ve Grand Total')
plt.xlabel('Value Type')
plt.ylabel('Miktar TL')

# Görseli gösterelim
plt.tight_layout()
plt.show()

##### Sales Time Analysis

In [ ]:
df_entegra_select_sales

In [ ]:
df_sales_time_select = df_entegra_select_sales[["date","time","total_product_quantity","total","tax","grand_total"]]
df_sales_time_select.head(5)

In [ ]:
df_sales_time_select.loc[:,"month"] = pd.to_datetime(df_sales_time_select.loc[:,"date"], format='%Y-%m-%d').dt.month
df_sales_time_select.loc[:,"hour"] = pd.to_datetime(df_sales_time_select.loc[:,"time"], format='%H:%M:%S').dt.hour
df_sales_time_select.head(5)

In [ ]:
df_sales_time_month = df_sales_time_select.groupby("month")[["total_product_quantity","total","tax","grand_total"]].sum().reset_index()
df_sales_time_month

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)

# graph = sns.barplot(data=df_entegra_select_sales, x=pd.to_datetime(df_entegra_select_sales["date"]).dt.month, y='total', estimator="sum")
graph = sns.barplot(data=df_sales_time_month, x='month', y='total', color='green')

# Grafik başlığı ve eksen etiketlerini ekleyelim
plt.title('Aylık Toplam Satış')
plt.xlabel('Ay')
plt.ylabel('Miktar TL')
graph.bar_label(ax.containers[0])

# Görseli gösterelim
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)

# graph = sns.barplot(data=df_entegra_select_sales, x=pd.to_datetime(df_entegra_select_sales["date"]).dt.month, y='total', estimator="sum")
graph = sns.barplot(data=df_sales_time_month, x='month', y='tax', color='red')

# Grafik başlığı ve eksen etiketlerini ekleyelim
plt.title('Aylık Satış Vergi')
plt.xlabel('Ay')
plt.ylabel('Miktar TL')
graph.bar_label(ax.containers[0])

# Görseli gösterelim
plt.tight_layout()
plt.show()

In [ ]:
df_sales_time_hour = df_sales_time_select.groupby("hour")[["total","tax","grand_total"]].sum().reset_index()
df_sales_time_hour

In [ ]:
fig, ax = plt.subplots(figsize=(16,9), dpi=100)

# graph = sns.barplot(data=df_entegra_select_sales, x=pd.to_datetime(df_entegra_select_sales["date"]).dt.month, y='total', estimator="sum")
graph = sns.pointplot(data=df_sales_time_hour, x='hour', y='total', color='green')

# Grafik başlığı ve eksen etiketlerini ekleyelim
plt.title('Saatlik Toplam Satış')
plt.xlabel('Hour')
plt.ylabel('Miktar TL')

# Görseli gösterelim
plt.tight_layout()
plt.show()